# 03. Inferência e Demo Interativa

Este notebook demonstra como carregar modelos treinados e criar uma interface interativa (Gradio) para detecção de deepfakes em tempo real.

In [ ]:
# 1. Configuração do Ambiente
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    if not os.path.exists('/content/TCC'):
        !git clone https://github.com/thieryw/TCC.git /content/TCC
    os.chdir('/content/TCC')
    !pip install -r requirements.txt
    !pip install gradio
    sys.path.append('/content/TCC')
    
    # Montar Drive para acessar modelos salvos
    from app.utils.colab import mount_drive
    mount_drive()
else:
    if os.path.exists('app'):
        sys.path.append(os.getcwd())

## 2. Carregar Serviço de Detecção
Inicializa o `DetectionService` apontando para o diretório de modelos.

In [ ]:
from app.domain.services.detection_service import DetectionService
from app.domain.models.audio_data import AudioData
import librosa
import numpy as np

# Diretório de modelos (padrão ou Drive)
MODELS_DIR = "app/models"
if is_colab() and os.path.exists('/content/drive/MyDrive/TCC_Models'):
    MODELS_DIR = "/content/drive/MyDrive/TCC_Models"

print(f"Carregando modelos de: {MODELS_DIR}")
detection_service = DetectionService(models_dir=MODELS_DIR)

# Listar modelos disponíveis
available_models = detection_service.get_available_models()
print("Modelos disponíveis:", available_models)

## 3. Interface Gradio
Cria uma interface web simples para upload de áudio e teste.

In [ ]:
import gradio as gr

def predict_audio(audio_path, model_choice):
    if not audio_path:
        return "Nenhum áudio fornecido."
    
    try:
        # Carregar áudio com librosa
        y, sr = librosa.load(audio_path, sr=None)
        audio_data = AudioData(audio=y, sample_rate=sr)
        
        # Executar detecção
        result = detection_service.detect_single(audio_data, model_name=model_choice)
        
        if result.status.name == 'SUCCESS':
            data = result.data
            veredicto = "FAKE" if data.is_fake else "REAL"
            confianca = f"{data.confidence:.2%}"
            return {
                "Veredicto": veredicto,
                "Confiança": confianca,
                "Detalhes": str(data.metadata)
            }
        else:
            return f"Erro: {result.errors}"
            
    except Exception as e:
        return f"Erro processando áudio: {str(e)}"

# Se não houver modelos, criar opção dummy
if not available_models:
    model_choices = ["Nenhum modelo carregado"]
else:
    model_choices = available_models

iface = gr.Interface(
    fn=predict_audio,
    inputs=[
        gr.Audio(type="filepath", label="Upload de Áudio"),
        gr.Dropdown(choices=model_choices, label="Escolha o Modelo", value=model_choices[0] if model_choices else None)
    ],
    outputs="json",
    title="Deepfake Audio Detector",
    description="Faça upload de um arquivo de áudio para verificar se é deepfake."
)

# share=True cria um link público (útil no Colab)
iface.launch(share=True, debug=True)